In [ ]:
# 数据库结构

# predicting_road_accident_risk/                           【数据库】道路事故风险预测任务
# │
# ├── XGBoost_model                                        【Study】使用 XGBoost 进行事故风险预测的超参数优化记录
# └── LightGBM_model                                       【Study】使用 LightGBM 进行事故风险预测的超参数优化记录


# thermophysical_property_melting_point/                   【数据库】物理属性 - 熔点预测任务
# │
# └── XGBoost_model                                        【Study】使用 XGBoost 进行物质熔点预测的超参数优化记录



In [ ]:
# 数据库复制转移

import os

os.system('mysqldump -h 10.162.147.95 -u user1 -p123456 thermophysical_property_melting_point > dump.sql')
os.system('mysql     -h 10.162.147.95 -u user1 -p123456 -e "CREATE DATABASE Thermophysical_Property_Melting_Point;"')
os.system('mysql     -h 10.162.147.95 -u user1 -p123456 Thermophysical_Property_Melting_Point < dump.sql')


0

In [1]:
import os
import optuna
import socket
import pymysql
import pymysql




host = "10.162.147.95"
user = "user1"
password = "123456"


d:\Software\conda\envs\kaggle\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# 获取所有数据库列表

conn = pymysql.connect(
    host=host,
    user=user,
    password=password,
    charset="utf8mb4"
)

# === 系统库定义（禁止删除） ===
system_dbs = {
    "information_schema",
    "mysql",
    "performance_schema",
    "sys"
}


# === 获取并打印所有非系统数据库 ===
with conn.cursor() as cur:
    cur.execute("SHOW DATABASES;")
    dbs = [row[0] for row in cur.fetchall()]

    print("📦 当前非系统数据库列表：\n{}".format([db for db in dbs if db not in system_dbs]))
    for db in dbs:
        if db not in system_dbs:
            print(f" - {db}")

conn.close()

📦 当前非系统数据库列表：
['kaggle_melting_point_optuna', 'predicting_road_accident_risk', 'thermophysical_property_melting_point']
 - kaggle_melting_point_optuna
 - predicting_road_accident_risk
 - thermophysical_property_melting_point


In [13]:
# 想删除的数据库列表
# target_dbs = [    "kaggle_melting_point_optuna", "test_db1", "test_db2"     ]
target_dbs = [   "kaggle_melting_point_optuna"    ]

# === 系统库定义（禁止删除） ===
system_dbs = {
    "information_schema",
    "mysql",
    "performance_schema",
    "sys"
}

# === 连接数据库 ===
conn = pymysql.connect(
    host=host,
    user=user,
    password=password,
    charset="utf8mb4"
)

with conn.cursor() as cur:
    # 获取所有数据库
    cur.execute("SHOW DATABASES;")
    all_dbs = [row[0] for row in cur.fetchall()]
    print("📦 当前数据库列表：")
    for db in all_dbs:
        if db not in system_dbs:
            print(f" - {db}")


    print("\n🚀 开始检测与删除...")
    for db in target_dbs:
        if db in system_dbs:
            print(f"⛔ 跳过系统数据库：{db}")
        elif db not in all_dbs:
            print(f"⚠️ 数据库不存在：{db}")
        else:
            try:
                cur.execute(f"DROP DATABASE `{db}`;")
                print(f"✅ 已删除数据库：{db}")
            except Exception as e:
                print(f"❌ 删除失败 {db}：{e}")

conn.close()
print("\n🎯 操作完成。")


📦 当前数据库列表：
 - predicting_road_accident_risk
 - thermophysical_property_melting_point

🚀 开始检测与删除...
⚠️ 数据库不存在：kaggle_melting_point_optuna

🎯 操作完成。


In [7]:
# 打印指定 数据库 的 study名称
database_name = 'thermophysical_property_melting_point'  # 数据库名称

storage = f"mysql+pymysql://{user}:{password}@{host}:3306/{database_name}"

studies = optuna.study.get_all_study_summaries(storage=storage)
print("现有 study：", [s.study_name for s in studies])

for s in studies:
    study = optuna.load_study(study_name=s.study_name, storage=storage)
    print(f"Study: {s.study_name:50s}, Trials: {len(study.trials)}")

现有 study： ['XGBoost_model']
Study: XGBoost_model                                     , Trials: 33


In [ ]:
# 修改指定 数据库 的 study名称

database_name = 'predicting_road_accident_risk'  # 数据库名称

# 连接数据库
conn = pymysql.connect(
    host=host,
    user=user,
    password=password,
    database=database_name,
    charset="utf8mb4"
)

cur = conn.cursor()

# 执行修改名称的 SQL
rename_map = {
    "optuna_task1"         : "XGBoost_model"
}


for old_name, new_name in rename_map.items():
    sql = "UPDATE studies SET study_name=%s WHERE study_name=%s"
    cur.execute(sql, (new_name, old_name))
    print(f"✅ 已将 '{old_name}' 重命名为 '{new_name}'")

conn.commit()
cur.close()
conn.close()


In [6]:
# 删除指定 数据库 的 study

database = "predicting_road_accident_risk"  # 目标数据库名称
# ========== 选择要删除的 study ==========
to_delete = ['optuna_task1', 'test', 'test2', 'test2xgboost']




storage = f"mysql+pymysql://{user}:{password}@{host}:3306/{database}"

# ========== 获取所有 study ==========
studies = optuna.study.get_all_study_summaries(storage=storage)
print("删除前存在的 study：")
for i, s in enumerate(studies, 1):
    print(f"{i}. {s.study_name}")
print("\n")


for name in to_delete:
    name = name.strip()
    if not name:
        continue

    try:
        optuna.delete_study(study_name=name, storage=storage)
        print(f"✅ 已删除：{name}")
    except Exception as e:
        print(f"❌ 删除 {name:50s} 失败：{e}")


# ========== 获取所有 study ==========
studies = optuna.study.get_all_study_summaries(storage=storage)
print("\n删除后存在的 study：")
for i, s in enumerate(studies, 1):
    print(f"{i}. {s.study_name}")


删除前存在的 study：
1. optuna_task1
2. test
3. test2
4. XGBoost_model
5. LightGBM_model
6. test2xgboost


✅ 已删除：optuna_task1
✅ 已删除：test
✅ 已删除：test2
✅ 已删除：test2xgboost

删除后存在的 study：
1. XGBoost_model
2. LightGBM_model
